In [2]:
from dotenv import load_dotenv
load_dotenv() # load GOOGLE_API_KEY from .env file

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        max_tokens=None,
    )

# RDFs Limitations
Notice, in the .ttl file, how we need to create inverse relation (killed_by)
and create 2 married relations (Joe -> Love) and (Love -> Joe)

In [6]:
from langchain_community.graphs import RdfGraph
graph = RdfGraph(
    source_file="rdfs_graph.ttl",
    standard="rdfs",
)

graph.load_schema()
graph.get_schema

'In the following, each IRI is followed by the local name and optionally its description in parentheses. \nThe RDF graph supports the following node types:\n<http://xmlns.com/ex/0.1/Male> (Male, None), <http://xmlns.com/ex/0.1/Person> (Person, None), <http://xmlns.com/ex/0.1/Fictional> (Fictional, None), <http://xmlns.com/ex/0.1/Female> (Female, None), <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property> (Property, None)\nThe RDF graph supports the following relationships:\n<http://xmlns.com/ex/0.1/married> (married, None), <http://xmlns.com/ex/0.1/killed> (killed, None), <http://xmlns.com/ex/0.1/killed_by> (killed_by, None)\n'

In [7]:
from langchain.chains import GraphSparqlQAChain
chain = GraphSparqlQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True, return_direct=False
)


In [9]:
res = chain.invoke("Is Love married? hint: use select query.\nDON'T surround the query with backticks")
# res = chain.invoke("Is Love alive?"
#                    "\nDON'T surround the query with backticks")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX ex: <http://xmlns.com/ex/0.1/>
SELECT ?spouse
WHERE {
    ex:Love ex:married ?spouse .
}
Full Context:
[(rdflib.term.URIRef('http://xmlns.com/ex/0.1/JoeGoldberg'),)]

> Finished chain.


In [10]:
res["result"]

'Based on the information provided, yes, Love is married to Joe Goldberg.'